In [25]:
!pip install datasets

In [26]:
!pip install country_list


Follow this [link](https://pypi.org/project/country_list/) to learn more about the python list of countries

In [27]:
from datasets import load_dataset

dataset2 = load_dataset("dutta18/omcs_dataset_of_commonsense_facts")

In [28]:
# Extracting sentences from wikitext; we did not end up working with them as
# they are not common sense ones

# from datasets import load_dataset

# dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

In [29]:
print(dataset2)

DatasetDict({
    train: Dataset({
        features: ['fact', 'count'],
        num_rows: 1578238
    })
})


In [30]:
print(dataset2['train'][1]['fact'])
print(dataset2['train'][1]['count'])

chose is a translation of thing.
2


In [31]:
# Get all the countries we are going to look for
from country_list import countries_for_language

country_tuples = countries_for_language('en')
print(country_tuples)
country_names = []
for abb, name in country_tuples:
  country_names.append(name)
print('Number of python countries', len(country_names))
print(country_names)


[('AF', 'Afghanistan'), ('AX', 'Åland Islands'), ('AL', 'Albania'), ('DZ', 'Algeria'), ('AS', 'American Samoa'), ('AD', 'Andorra'), ('AO', 'Angola'), ('AI', 'Anguilla'), ('AQ', 'Antarctica'), ('AG', 'Antigua & Barbuda'), ('AR', 'Argentina'), ('AM', 'Armenia'), ('AW', 'Aruba'), ('AU', 'Australia'), ('AT', 'Austria'), ('AZ', 'Azerbaijan'), ('BS', 'Bahamas'), ('BH', 'Bahrain'), ('BD', 'Bangladesh'), ('BB', 'Barbados'), ('BY', 'Belarus'), ('BE', 'Belgium'), ('BZ', 'Belize'), ('BJ', 'Benin'), ('BM', 'Bermuda'), ('BT', 'Bhutan'), ('BO', 'Bolivia'), ('BA', 'Bosnia & Herzegovina'), ('BW', 'Botswana'), ('BV', 'Bouvet Island'), ('BR', 'Brazil'), ('IO', 'British Indian Ocean Territory'), ('VG', 'British Virgin Islands'), ('BN', 'Brunei'), ('BG', 'Bulgaria'), ('BF', 'Burkina Faso'), ('BI', 'Burundi'), ('KH', 'Cambodia'), ('CM', 'Cameroon'), ('CA', 'Canada'), ('CV', 'Cape Verde'), ('BQ', 'Caribbean Netherlands'), ('KY', 'Cayman Islands'), ('CF', 'Central African Republic'), ('TD', 'Chad'), ('CL', '

In [35]:
# Not using regex, but does not work all the time
#         for country in country_names:
#             if country.lower() in stripped_sentence.lower():
#                 # print(stripped_sentence)
#                 # print(country)
#                 filtered_sentences.append(stripped_sentence)

import re

# Filted through the common sense dataset to have only the sentences with
# countries words
def find_country_sentences(text, country_names):
    # print('text', text)
    sentences = text.split(".")
    filtered_sentences = []
    # print('sentence', sentences)

    country_pattern = r'\b(' + '|'.join(re.escape(country) for country in country_names) + r')\b'

    for sentence in sentences:
        stripped_sentence = sentence.strip()
        if re.search(country_pattern, stripped_sentence, re.IGNORECASE):
            filtered_sentences.append(stripped_sentence)

    return filtered_sentences

In [33]:
# In the filtered sentence, count the occurence of each word
def count_top(text, countries):
  hash = {country: 0 for country in countries}

  for sentence in text:
    for country in countries:
      if country.lower() in sentence.lower():
        hash[country] +=1

  return hash

In [36]:
facts = []
for entry in dataset2['train']:
    text = entry['fact']
    facts.extend(find_country_sentences(text, country_names))


print(facts[:10])

['apple of Peru is a synonym of jimsonweed', 'Krasnojarsk is part of Russia', 'Republic of South Africa is a synonym of South Africa', 'Brisbane is part of Australia', 'Most people in Japan speak Japanese', 'saint pierre and miquelon is part of france', 'picture description: mt zermatt in switzerland', 'The capital of Tuvalu is Funafuti', 'gun room is used in the region of United Kingdom', 'The statement "the statue of liberty comes from france" is true because The French made the Statue of Liberty']


In [37]:
counttemp = count_top(facts, country_names)
print(counttemp)

{'Afghanistan': 108, 'Åland Islands': 0, 'Albania': 21, 'Algeria': 34, 'American Samoa': 13, 'Andorra': 15, 'Angola': 20, 'Anguilla': 14, 'Antarctica': 125, 'Antigua & Barbuda': 0, 'Argentina': 88, 'Armenia': 40, 'Aruba': 10, 'Australia': 928, 'Austria': 98, 'Azerbaijan': 17, 'Bahamas': 27, 'Bahrain': 24, 'Bangladesh': 22, 'Barbados': 37, 'Belarus': 27, 'Belgium': 64, 'Belize': 15, 'Benin': 29, 'Bermuda': 72, 'Bhutan': 15, 'Bolivia': 30, 'Bosnia & Herzegovina': 0, 'Botswana': 28, 'Bouvet Island': 7, 'Brazil': 180, 'British Indian Ocean Territory': 7, 'British Virgin Islands': 10, 'Brunei': 20, 'Bulgaria': 27, 'Burkina Faso': 12, 'Burundi': 19, 'Cambodia': 23, 'Cameroon': 22, 'Canada': 595, 'Cape Verde': 23, 'Caribbean Netherlands': 0, 'Cayman Islands': 17, 'Central African Republic': 28, 'Chad': 46, 'Chile': 64, 'China': 769, 'Christmas Island': 17, 'Cocos (Keeling) Islands': 8, 'Colombia': 50, 'Comoros': 12, 'Congo - Brazzaville': 0, 'Congo - Kinshasa': 0, 'Cook Islands': 8, 'Costa Ri

In [38]:
import csv

with open('count.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in counttemp.items():
        writer.writerow([key, value])


In [39]:
# Sort the country list by the max number of occurence in the filtered sentences
sorted_list = sorted(counttemp, key=counttemp.get, reverse=True)

In [40]:
# Work with only the top 50 most commonly mentioned countries
# Replace the places in those 50 that are not countries with the next ones
# in the ordered list that are countries
print(sorted_list[:50])
res = [sub.replace('Bermuda', 'Netherlands') for sub in sorted_list[:50]]
res = [sub.replace('Jersey', 'Belgium') for sub in res]
res = [sub.replace('Antarctica', 'Chile') for sub in res]

['United States', 'Australia', 'China', 'France', 'United Kingdom', 'Canada', 'Japan', 'Mexico', 'Germany', 'India', 'Italy', 'Jersey', 'Israel', 'Egypt', 'Turkey', 'Russia', 'Iraq', 'Spain', 'Jordan', 'Sierra Leone', 'Brazil', 'Ireland', 'Georgia', 'Guinea', 'Greece', 'Antarctica', 'New Zealand', 'Zimbabwe', 'Switzerland', 'Afghanistan', 'Iran', 'Austria', 'Cuba', 'Saudi Arabia', 'Niger', 'Poland', 'Argentina', 'Philippines', 'Norway', 'Portugal', 'Finland', 'Nepal', 'Vietnam', 'Bermuda', 'Sweden', 'Oman', 'Pakistan', 'Panama', 'Denmark', 'South Africa']


In [41]:
print(res)

['United States', 'Australia', 'China', 'France', 'United Kingdom', 'Canada', 'Japan', 'Mexico', 'Germany', 'India', 'Italy', 'Belgium', 'Israel', 'Egypt', 'Turkey', 'Russia', 'Iraq', 'Spain', 'Jordan', 'Sierra Leone', 'Brazil', 'Ireland', 'Georgia', 'Guinea', 'Greece', 'Chile', 'New Zealand', 'Zimbabwe', 'Switzerland', 'Afghanistan', 'Iran', 'Austria', 'Cuba', 'Saudi Arabia', 'Niger', 'Poland', 'Argentina', 'Philippines', 'Norway', 'Portugal', 'Finland', 'Nepal', 'Vietnam', 'Netherlands', 'Sweden', 'Oman', 'Pakistan', 'Panama', 'Denmark', 'South Africa']


ONLY TOP 50
-- now extract the sentences that contain the top 50 countries only

In [42]:
facts = []
res = ['United States', 'Australia', 'China', 'France', 'United Kingdom', 'Canada', 'Japan', 'Mexico', 'Germany', 'India', 'Italy', 'Belgium', 'Israel', 'Egypt', 'Turkey', 'Russia', 'Iraq', 'Spain', 'Jordan', 'Sierra Leone', 'Brazil', 'Ireland', 'Georgia', 'Guinea', 'Greece', 'Chile', 'New Zealand', 'Zimbabwe', 'Switzerland', 'Afghanistan', 'Iran', 'Austria', 'Cuba', 'Saudi Arabia', 'Niger', 'Poland', 'Argentina', 'Philippines', 'Norway', 'Portugal', 'Finland', 'Nepal', 'Vietnam', 'Netherlands', 'Sweden', 'Oman', 'Pakistan', 'Panama', 'Denmark', 'South Africa']
for entry in dataset2['train']:
    text = entry['fact']
    facts.extend(find_country_sentences(text, res))


print(facts[:10])

# The total number of filtered sentences that we manually looked through
# We ended up using only 400 of them
print(len(facts))

['Krasnojarsk is part of Russia', 'Republic of South Africa is a synonym of South Africa', 'Brisbane is part of Australia', 'Most people in Japan speak Japanese', 'saint pierre and miquelon is part of france', 'picture description: mt zermatt in switzerland', 'gun room is used in the region of United Kingdom', 'The statement "the statue of liberty comes from france" is true because The French made the Statue of Liberty', 'The canada goose is a large north american bird that honks', 'Portuguese Republic is a synonym of Portugal']
11930


As we can see we have 11930 samples for the top 50 countries that we want to use for our testing, but after filtering through them, most of the senteces were removed, and we ended up with 350 samples from the omcs.

**These are a few of the examples of the removed sentence:**

japan is an island in the pacific

josh flew to israel

you are likely to find a temple in china

rates is used in the region of united kingdom

you are not likely to find a marmot in philippines

you are likely to find a mouse in italy

situation: i am in germany

carriageway is used in the region of united kingdom

to understand the event "judy learned about japan

the united states is only one country in the world

In [43]:
# Get the count of every county in the selected top 50 countries sentences
counttemp_50 = count_top(facts, res)
print(counttemp_50)
import csv

with open('count_50.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in counttemp_50.items():
        writer.writerow([key, value])


{'United States': 1836, 'Australia': 925, 'China': 765, 'France': 622, 'United Kingdom': 598, 'Canada': 595, 'Japan': 520, 'Mexico': 457, 'Germany': 414, 'India': 372, 'Italy': 377, 'Belgium': 64, 'Israel': 315, 'Egypt': 310, 'Turkey': 294, 'Russia': 255, 'Iraq': 189, 'Spain': 186, 'Jordan': 185, 'Sierra Leone': 184, 'Brazil': 180, 'Ireland': 178, 'Georgia': 158, 'Guinea': 145, 'Greece': 137, 'Chile': 64, 'New Zealand': 112, 'Zimbabwe': 111, 'Switzerland': 110, 'Afghanistan': 108, 'Iran': 101, 'Austria': 97, 'Cuba': 94, 'Saudi Arabia': 93, 'Niger': 57, 'Poland': 91, 'Argentina': 88, 'Philippines': 88, 'Norway': 87, 'Portugal': 80, 'Finland': 77, 'Nepal': 76, 'Vietnam': 74, 'Netherlands': 66, 'Sweden': 72, 'Oman': 36, 'Pakistan': 68, 'Panama': 68, 'Denmark': 67, 'South Africa': 67}


In [44]:
# Export the collected dataset
import string

with open('country_facts.txt', 'w') as f:
    for fact in facts:
        fact = fact.replace(string.punctuation, "")
        fact = fact.lower()
        f.write(f"%s\n" % fact)
